## gensim으로 네이버 기사 토픽 모델링 해보기

> 토픽 모델링을 적용하기 위해 텍스트를 처리합니다.

> 토픽 모델링 라이브러리인 gensim을 사용해봅니다.

In [1]:
# !pip install gensim

### 1. 토픽 모델링을 위한 라이브러리 불러오기

In [2]:
from tqdm import tqdm_notebook # progress bar
import MeCab # Mecab, Okt 등 형태소 분석기 불러오기
import string # 특수문자
import warnings # 경고 알림 제거를 위한 라이브러리
from gensim import corpora # gensim에서 사용하는 vectorizer 모듈과, LDA model을 불러온다.
from gensim import models


import numpy as np
import re
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings("ignore", category=DeprecationWarning) # 경고 알림이 뜨면 모두 무시합니다.

In [3]:
mecab = MeCab.Tagger()

In [4]:
import re

def mecab_nouns(text):
    nouns = []
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile('.*\t[A-Z]+')
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split('\t')) for token in mecab.parse(text).splitlines()[:-1]]    
        
    # 추출한 token중에 POS가 명사 분류에 속하는 토큰만 선택.
    for token in temp:
        if re.match('N[A-Z]+', token[1]):
            nouns.append(token[0])
    return nouns

def mecab_morphs(text):
    morphs = []
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    pattern = re.compile(".*\t[A-Z]+")
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    temp = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]] 
    
    # 추출한 token중에 문자열만 선택.
    for token in temp:
        morphs.append(token[0])
    
    return morphs

def mecab_pos(text):
    pos = []
    
    # 우리가 원하는 TOKEN\tPOS의 형태를 추출하는 정규표현식.
    # re()보다 re.compile()로 해서 미리 컴파일해두면 속도가 더 빠름
    pattern = re.compile(".*\t[A-Z]+") # 토큰이름, 탭, 품사종류
    
    # 패턴에 맞는 문자열을 추출하여 konlpy의 mecab 결과와 같아지도록 수정.
    # group(0) -> 생성된 객체에서 스트링만 뽑아줌
    pos = [tuple(pattern.match(token).group(0).split("\t")) for token in mecab.parse(text).splitlines()[:-1]]
    
    return pos

### 2. 텍스트 전처리 함수 만들기

In [5]:
def read_documents(input_file_name):
    corpus = []

    # pk 파일을 읽어서 리스트로 변환하여 돌려줌.
    with open(input_file_name, 'rb') as f:
        temp_corpus = pickle.load(f)
    for page in temp_corpus:
        corpus += page
    return corpus

def text_cleaning(docs):
    # 한국어를 제외한 글자를 제거하는 함수.
    cleaned_docs = []
    
    for doc in docs:
        temp_doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)
        cleaned_docs.append(temp_doc)
    return cleaned_docs

def define_stopwords(path):
    SW = set()
    for i in string.punctuation:  # 특수문자 추가해줌
        SW.add(i)
        
    with open(path, encoding="utf-8") as f: ## 파일 작업이 끝나면 파일을 자동으로 닫아줌
        for word in f:
            SW.add(word)
    return SW

def text_tokenizing(corpus, tokenizer):
    # 명사 추출 / 형태소 분석 두가지를 선택할 수 있게 만들어주는 함수를 만들어보자
    mecab = MeCab.Tagger()
    token_corpus = []
    # tqdm을 사용하여 진행과정을 볼 수 있게 하자
    if tokenizer == "noun":
        for n in tqdm_notebook(range(len(corpus)), desc="preprocessing!!!"):
            token_text = mecab_nouns(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)
    elif tokenizer == "morph":
        for n in tqdm_notebook(range(len(corpus)), desc="preprocessing!!!!"):
            token_text = mecab_morphs(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)
    elif tokenizer == "word":
        for n in tqdm_notebook(range(len(corpus)), dec = "preprocessing!!"):
            token_text = corpus[n].split()
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)
    return token_corpus

In [6]:
# 함수를 불러오는 (메인) 코드.
input_file_name = "./data/naver_news_content.pk"
documents = read_documents(input_file_name)
SW = define_stopwords("./data/stopwords-ko.txt")
cleaned_text = text_cleaning(documents)
tokenized_text = text_tokenizing(cleaned_text, tokenizer="noun") #tokenizer= "noun" or "morph" or "word"

문서 읽기의 과정은 앞서 단어 임베딩의 경우와 다르지 않다. 다음 과정은 문서-단어 행렬을 만드는 과정이다.

In [7]:
# 결과 확인.
print(tokenized_text[0])

['포항', '환경일보', '김용달', '기자', '포스코', '진행', '창업', '지원', '프로그램', '취업', '준비', '사이', '호평', '최근', '택트', '교육', '방식', '프로그램', '도입', '교육', '창업', '사례', '혁신', '성과', '마리', '토끼', '평가', '포스코', '취업', '아카데미', '화상', '회의', '택트', '교육', '진행', '모습', '사진', '제공', '포스코', '문단', '좌측', '추가', '포스코', '청년', '인재', '육성', '취업', '창업', '지원', '포유', '드림', '교육', '운영', '목표', '창업', '실질', '도움', '콘텐츠', '교육', '제공', '포유', '프로그램', '포스코', '취업', '아카데미', '청년', '아카데미', '창업', '스쿨', '구성', '포스코', '취업', '아카데미', '취업', '경쟁력', '실무', '역량', '강화', '참여', '실무', '과제', '수행', '교육', '프로그램', '포스코', '코로나', '사태', '순연', '프로그램', '최근', '화상', '회의', '택트', '교육', '방식', '도입', '주간', '교육', '기간', '동안', '주차', '온라인', '기업', '직무', '분석', '자기소개', '코칭', '모의', '면접', '진행', '주차', '합숙', '교육', '창의', '문제', '해결', '방법론', '바탕', '조별', '과제', '수행', '진행', '현재', '포스코', '취업', '아카데미', '수료', '취업', '성공', '지난해', '포스코', '취업', '안동환', '교육', '취업', '준비', '과정', '도움', '기업', '업무', '문제', '해결', '방법', '이해', '포스코', '취업', '아카데미', '포항', '지역', '진행', '교육', '지원자', '모집', '모집', '기간', '홈페이지', '신청', '포스코', '교육

In [8]:
print(tokenized_text[1])

['환경일보', '우리', '지역', '직업훈련', '취업', '고용', '노동부', '국민', '카드', '취업률', '상위', '직종', '훈련', '과정', '공개', '서울', '경기', '인천', '강원', '대전', '대구', '광주', '부산', '권역', '취업률', '상위', '직종', '훈련', '과정', '문단', '좌측', '추가', '권역', '취업률', '직종', '서울', '법률', '정보', '기술', '직종', '경기', '인천', '기계가공', '직종', '대전', '정보', '기술', '직종', '강원', '광주', '대구', '의료', '직종', '부산', '자동차', '직종', '정보', '기술', '분야', '경우', '대전', '서울', '광주', '다수', '지역', '상위', '직종', '포함', '디지털', '기술', '분야', '인력', '수요', '증가', '주요', '훈련', '대상', '청년', '여성', '장년', '훈련', '직종', '취업률', '훈련', '직종', '청년', '훈련', '직종', '커피', '바리스타', '과정', '식음료', '조리', '직종', '데이터', '활용', '자바', '파이썬', '개발자', '양성', '정보', '기술', '직종', '취업률', '직종', '법률', '사무', '취업', '과정', '법률', '직종', '컴퓨터', '응용', '기계', '과정', '기계', '가공', '직종', '여성', '경우', '훈련', '직종', '요양', '보호사', '자격', '취득', '과정', '보건', '직종', '취업률', '직종', '법률', '사무', '취업', '과정', '법률', '직종', '중장', '경우', '훈련', '직종', '요양', '보호사', '자격', '취득', '보건', '직종', '취업률', '직종', '기계설계', '제작', '실무', '기계', '가공', '직종', '자료', '제공', '고용', '노동부', '국민', '카드', '국민',

### 3. 토픽 모델링에 사용할 함수들 확인하기

In [9]:
# 문서-단어 행렬 만들기
# 어휘(vocabulary) 학습
dictionary = corpora.Dictionary(tokenized_text)

# 문서-단어 행렬(document-term matrix) 생성
corpus = [dictionary.doc2bow(text) for text in tokenized_text] # bow = bag of word
# 여기서 text = 기사 한개

In [10]:
# Dictionary 확인
print(dictionary)

Dictionary(284 unique tokens: ['감염', '강사', '강의', '강화', '개월']...)


In [11]:
# corpus 확인
print(corpus[1]) ## (a,b)일때 a는 (dictionary에서 각 단어에 연결되는) 인덱스, b는 들어있는 word개수

[(9, 1), (11, 12), (12, 1), (18, 1), (20, 1), (22, 1), (23, 1), (28, 1), (30, 1), (32, 1), (41, 1), (42, 1), (44, 1), (50, 2), (51, 1), (55, 5), (57, 1), (66, 1), (69, 1), (71, 1), (76, 1), (77, 3), (83, 4), (89, 1), (90, 1), (92, 2), (98, 3), (105, 1), (109, 1), (110, 2), (112, 1), (119, 2), (120, 1), (125, 1), (131, 4), (133, 1), (134, 4), (135, 1), (140, 1), (150, 1), (157, 2), (158, 1), (160, 2), (161, 2), (162, 1), (163, 1), (164, 1), (165, 1), (166, 2), (167, 3), (168, 1), (169, 4), (170, 1), (171, 1), (172, 3), (173, 1), (174, 1), (175, 6), (176, 1), (177, 2), (178, 1), (179, 3), (180, 1), (181, 1), (182, 2), (183, 1), (184, 6), (185, 1), (186, 1), (187, 1), (188, 2), (189, 1), (190, 1), (191, 1), (192, 2), (193, 2), (194, 3), (195, 2), (196, 6), (197, 1), (198, 1), (199, 1), (200, 1), (201, 1), (202, 1), (203, 5), (204, 2), (205, 2), (206, 2), (207, 1), (208, 1), (209, 2), (210, 1), (211, 4), (212, 3), (213, 1), (214, 1), (215, 3), (216, 1), (217, 1), (218, 1), (219, 1), (220, 

In [12]:
# TFIDF 문서-단어 행렬 생성
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
print(corpus_tfidf[0])

[(0, 0.02823912473624525), (1, 0.0564782494724905), (2, 0.02823912473624525), (3, 0.02823912473624525), (4, 0.02823912473624525), (5, 0.02823912473624525), (6, 0.02823912473624525), (7, 0.0564782494724905), (8, 0.02823912473624525), (10, 0.02823912473624525), (13, 0.6494998689336408), (14, 0.02823912473624525), (15, 0.02823912473624525), (16, 0.02823912473624525), (17, 0.02823912473624525), (19, 0.02823912473624525), (21, 0.02823912473624525), (24, 0.02823912473624525), (25, 0.0564782494724905), (26, 0.02823912473624525), (27, 0.0564782494724905), (29, 0.02823912473624525), (31, 0.0564782494724905), (33, 0.0564782494724905), (34, 0.02823912473624525), (35, 0.02823912473624525), (36, 0.02823912473624525), (37, 0.02823912473624525), (38, 0.02823912473624525), (39, 0.02823912473624525), (40, 0.0564782494724905), (43, 0.02823912473624525), (45, 0.0564782494724905), (46, 0.02823912473624525), (47, 0.02823912473624525), (48, 0.02823912473624525), (49, 0.02823912473624525), (52, 0.02823912473

In [13]:
# LDA model 만들기
model = models.ldamodel.LdaModel(corpus, num_topics=3, id2word=dictionary)

In [14]:
# LDA 결과 확인
# model.show_topic(topic_id, num_words)
model.show_topic(2, 10) 

[('직종', 0.04561953),
 ('훈련', 0.027452271),
 ('과정', 0.022799302),
 ('취업', 0.015663952),
 ('국민', 0.014077306),
 ('취업률', 0.01387467),
 ('교육', 0.01347322),
 ('기술', 0.012133128),
 ('디지털', 0.011535047),
 ('분야', 0.011490233)]

#### LDA의 결과!!
- 위에서 topic 3가지 만들었으므로 topic은 0번, 1번, 2번.. 이렇게 세개가 있다
- 각 topic마다 모든 단어가 해당 topic일 확률이 결과로 나온다
- 그중, 확률이 높은것을 몇개씩 고르면, 단어를 topic별로 분류한 것이다
- 이 때, 1번 토픽에 속한 단어가 2번 토픽에도 속할 수 있다

### 4. 토픽 모델링을 추가하여 코드 완성하기

In [15]:
# 토픽 개수, 키워드 개수를 정해주는 변수를 추가.
NUM_TOPICS = 3       # 토픽 개수
NUM_TOPIC_WORD = 30  # 해당 토픽일 확률이 가장 높은 30개를 해당 토픽으로 선별할것

def build_doc_term_mat(documents):
    # 문서-단어 행렬 만들어주는 함수.
    print("Building document-term matrix.")
    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
    return corpus, dictionary

def print_topic_words(model):
    # 토픽 모델링 결과를 출력해 주는 함수.
    print("\nPrinting topic words.\n")
    for topic_id in range(model.num_topics): ## model.num_topics = 3... 토픽개수
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORD)
        print("Topic ID: {}".format(topic_id))
        for topic_word, probs in topic_word_probs:
            print("\t{}\t{}".format(topic_word, probs))
        print('\n')


In [16]:
# document-term matrix를 만들고,
corpus, dictionary = build_doc_term_mat(tokenized_text)
# LDA를 실행.
model = models.ldamodel.LdaModel(corpus, 
                                 num_topics = NUM_TOPICS, 
                                 id2word = dictionary, 
                                 eta = 'auto', 
                                 alpha = 'auto')
# 결과를 출력.
print_topic_words(model)

Building document-term matrix.

Printing topic words.

Topic ID: 0
	직종	0.05651215836405754
	훈련	0.02890213206410408
	과정	0.024329829961061478
	취업	0.014120694249868393
	취업률	0.013641892932355404
	국민	0.01328736450523138
	카드	0.013121623545885086
	기술	0.01304259430617094
	디지털	0.012841952033340931
	분야	0.011588823981583118
	법률	0.010358596220612526
	핵심	0.01023243460804224
	청년	0.009558484889566898
	정보	0.009393821470439434
	직업훈련	0.009333382360637188
	양성	0.00867740623652935
	인재	0.00852796621620655
	산업	0.00849191565066576
	실무	0.008383693173527718
	고용	0.00793442688882351
	기계	0.007566120009869337
	경우	0.0075651840306818485
	광주	0.007372662425041199
	대전	0.007192994002252817
	서울	0.006853313650935888
	제공	0.006583596579730511
	교육	0.006492868531495333
	상위	0.006476552225649357
	방식	0.006151433102786541
	환경일보	0.005966389086097479


Topic ID: 1
	교육	0.046053070574998856
	취업	0.026934469118714333
	포스코	0.026007166132330894
	프로그램	0.020102979615330696
	진행	0.01833031326532364
	창업	0.018110288307070732
	직종	0.0151156401261

### 5. pyLDAvis를 통한 토픽 모델링 결과 시각화하기

In [17]:
# !pip install pyldavis

In [18]:
# pyLDAvis 불러오기
import pyLDAvis
import pyLDAvis.gensim

# pyLDAvis를 jupyter notebook에서 실행할 수 있게 활성화.
pyLDAvis.enable_notebook()

# pyLDAvis 실행.
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0     -0.050092  0.0       1        1  50.143528
1      0.046959  0.0       2        1  49.713863
2      0.003133  0.0       3        1   0.142596, topic_info=    Term       Freq      Total Category  logprob  loglift
13    교육  16.000000  16.000000  Default  30.0000  30.0000
262   직종  22.000000  22.000000  Default  29.0000  29.0000
134   취업  12.000000  12.000000  Default  28.0000  28.0000
142  포스코   9.000000   9.000000  Default  27.0000  27.0000
11    과정  10.000000  10.000000  Default  26.0000  26.0000
..   ...        ...        ...      ...      ...      ...
251   정보   0.004027   3.791780   Topic3  -5.4027  -0.2946
196  디지털   0.004305   5.437183   Topic3  -5.3359  -0.5883
169   고용   0.003961   3.704824   Topic3  -5.4192  -0.2879
203   법률   0.004050   4.578422   Topic3  -5.3971  -0.4775
267   카드   0.004004   5.488826   Topic3  -5.4084  -0.6702

[192 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
0         2  0.811104    감염
1         2  1.018814    강사
5         2  0.814235    거리
166       1  0.942464    경기
166       2  0.471232    경기
...     ...       ...   ...
157       1  0.563670  환경일보
157       2  0.563670  환경일보
159       2  0.508983    회의
283       1  0.750355    훈련
283       2  0.250118    훈련

[183 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])